In [1]:
from datasets import load_dataset

In [2]:
# ローカル環境からデータセットをインポート

datasets = load_dataset(
    'text',
    data_files = {
        "train": "train.txt",
        "validation": "validation.txt"
    }
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [3]:
model_name = "cyberagent/calm2-7b-chat"

In [4]:
from transformers import AutoTokenizer

In [5]:
tokenizer_false = AutoTokenizer.from_pretrained(
    model_name,
    use_fast = False
)

tokenizer_config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/585 [00:00<?, ?B/s]

In [6]:
%%time
a = tokenizer_false(datasets["train"][0]['text'])

CPU times: user 7.27 ms, sys: 12.2 ms, total: 19.5 ms
Wall time: 8.65 ms


In [7]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast = True
)

In [8]:
%%time
b = tokenizer(datasets["train"][0]['text'])

CPU times: user 6.39 ms, sys: 1.02 ms, total: 7.41 ms
Wall time: 5.27 ms


↑
スピード比較

use_fast = Trueの方が僅かに速い？？

use_fastにすることで正確性を欠いたという報告もあり注意（[参考](https://github.com/huggingface/text-generation-inference/issues/469)）

In [9]:
## 各データ単位のトークン化

def tokenize_function(examples):
    return tokenizer(examples['text'])

In [10]:
tokenized_datasets = datasets.map(
    tokenize_function,
    batched = True, # トークン化の処理に複数バッチ使う
    num_proc = 4, # 並行プロセス数
    remove_columns = ['text'] # textデータは用済みなので削除
)

Map (num_proc=4):   0%|          | 0/86 [00:00<?, ? examples/s]

num_proc must be <= 3. Reducing num_proc to 3 for dataset of size 3.


Map (num_proc=3):   0%|          | 0/3 [00:00<?, ? examples/s]

#### batched: バッチを使ったトークン化

データセットを特定のメモリサイズに落とし込む = 長い文章を短く分割する

大きめのバッチサイズを使うと処理が早く終わる傾向

ここでメモリサイズとはメモリを指すこともあるし、GPUを指すこともある

#### num_proc: マルチコア環境の場合

マルチコア環境 = 1つのCPUの中に複数の単位CPUがあるコンピュータ

マルチコア環境で実行している場合は、指定した数のCPUを使用する

並行処理することで高速化に寄与

コアの数よりも多く指定してしまうと、上限まで落とされる(警告が出る)

In [11]:
# ファインチューニング中に使用するメモリ領域を圧迫しないように、適切なサイズに文章をカットする

block_size = 128

In [12]:
def split_texts(examples):
    result = {
        'input_ids': [],
        'attention_mask': []
    }
    for ix in range(len(examples['input_ids'])):
        # 切り捨てを考慮してデータに含めるトークンIDsの数
        # カットサイズからはみ出る分のトークンIDは切り捨ててしまう、もったいないが
        total_length = len(examples['input_ids'][ix])
        total_length = (total_length // block_size) * block_size

        for bookmark in range(0, total_length, block_size):
            result['input_ids'].append(examples['input_ids'][ix][bookmark: bookmark + block_size])
            result['attention_mask'].append(examples['attention_mask'][ix][bookmark: bookmark + block_size])
    result["labels"] = result['input_ids'].copy() # transformersのファインチューニングのアルゴリズムによりlabelsとinput_idsが同じで問題ない(後述)
    return result

In [13]:
lm_datasets = tokenized_datasets.map(
    split_texts,
    batched = True,
    batch_size = 1000,
    num_proc = 4
)

Map (num_proc=4):   0%|          | 0/86 [00:00<?, ? examples/s]

num_proc must be <= 3. Reducing num_proc to 3 for dataset of size 3.


Map (num_proc=3):   0%|          | 0/3 [00:00<?, ? examples/s]

#### labelsとinput_idsが同じで問題ない

hugging-faceのライブラリでファインチューニングする場合、labelsとinput_idsが同じで問題ない

実際のチューニングでは、1つのlabelsとinput_idsの組がなんども再利用される
というのも実際のチューニングアルゴリズムによるもので

途中まで生成された文章から次のトークンを予測して答え合わせ、というのを文章の最後まで行っているため

だから一つ前のlabelsが次のinput_idsになるというように一つずつずらしている

これをtransformersが気を効かせて勝手にやってくれる

## ここからファインチューニング

In [14]:
from transformers import (
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer
)
import torch
from peft import LoraConfig

In [15]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map = 'auto', # 複数のGPUがあるときに、それらを均等に使用する
    load_in_8bit = True # 
)

config.json:   0%|          | 0.00/669 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.04G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [16]:
output_dir = './results'
evaluation_strategy = 'epoch'
learning_rate = 2e-5
weight_decay = 0.01

training_args = TrainingArguments(
    output_dir,
    evaluation_strategy = evaluation_strategy,
    learning_rate = learning_rate,
    weight_decay = weight_decay
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [17]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [18]:
model.add_adapter(peft_config)

In [19]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = lm_datasets["train"],
    eval_dataset = lm_datasets["validation"],
)

In [20]:
%%time
trainer.train()

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:1 and cuda:0!